In [1]:
# Requires clientside communication install of Azures Custom Vision API
# `pip install azure-cognitiveservices-vision-customvision`
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch
from msrest.authentication import ApiKeyCredentials 

import os
import numpy as np

In [2]:
# Azure Communication
ENDPOINT = "https://customid.cognitiveservices.azure.com/"
training_key = "7dff09b1d60e4fbb8c71305737d8c771"
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
publish_iteration_name = "customid"
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

project = trainer.create_project("customid")

In [3]:
os.chdir('./Bird_Train/train')

# Classify each species by the folder it's contained in
tags = [name for name in os.listdir('.') if os.path.isdir(name)]

FileNotFoundError: [Errno 2] No such file or directory: './Bird_Training/train'

In [ ]:
def createTag(tag_name):
    # Create a new tag in the Custom Vision project
    tag = trainer.create_tag(project.id, tag_name)
    return tag.id

def createImageList(tag, tag_id):
# Set directory to current tag.
   base_image_url = f"./{tag}/"
   photo_name_list = os.listdir(base_image_url)
   image_list = []
   for file_name in photo_name_list:
       with open(base_image_url+file_name, "rb") as image_contents:
           image_list.append(ImageFileCreateEntry(name=base_image_url+file_name, contents=image_contents.read(), tag_ids=[tag_id]))
   return image_list

def uploadImageList(image_list):
      upload_result = trainer.create_images_from_files(project_id=project.id, batch=image_list)
      if not upload_result.is_batch_successful:
         print("Image batch upload failed.")
         for image in upload_result.images:
              print("Image status: ", image.status)
         exit(-1)

In [ ]:
for tag in tags:
    tag_id = createTag(tag)
    print(f"tag creation done with tag id {tag_id}")
    image_list = createImageList(tag, tag_id)
    print("image_list created with length " + str(len(image_list)))

    # Depending on the ammount of transactions, the upload batch sizes should be changed. (Currently 50)
    for i in range(0, len(image_list), 25):
        batch = ImageFileCreateBatch(images=image_list[i:i + 25])
        print(f'Upload started for batch {i} total items {len(image_list)} for tag {tag}...')
        uploadImageList(batch)
        print(f"Batch {i} Image upload completed. Total uploaded {len(image_list)} for tag {tag}")